# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Feb 1 2023 9:15AM,256369,19,9103169,"Emersa Waterbox, LLC",Released
1,Feb 1 2023 9:09AM,256364,16,SHL-9102667,"SHL Pharma, LLC",Released
2,Feb 1 2023 9:05AM,256360,15,9097782,"Brookfield Pharmaceuticals, LLC",Released
3,Feb 1 2023 9:05AM,256360,15,9097783,"Brookfield Pharmaceuticals, LLC",Released
4,Feb 1 2023 9:05AM,256360,15,9097784,"Brookfield Pharmaceuticals, LLC",Released
5,Feb 1 2023 9:05AM,256360,15,9097785,"Brookfield Pharmaceuticals, LLC",Released
6,Feb 1 2023 9:05AM,256360,15,9097786,"Brookfield Pharmaceuticals, LLC",Released
7,Feb 1 2023 9:05AM,256360,15,9097787,"Brookfield Pharmaceuticals, LLC",Released
8,Feb 1 2023 9:05AM,256360,15,9097788,"Brookfield Pharmaceuticals, LLC",Released
9,Feb 1 2023 9:05AM,256360,15,9097789,"Brookfield Pharmaceuticals, LLC",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
38,256361,Released,1
39,256362,Released,15
40,256363,Released,15
41,256364,Released,1
42,256369,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
256361,NaN,NaN,1.0
256362,NaN,NaN,15.0
256363,NaN,NaN,15.0
256364,NaN,NaN,1.0
256369,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
256271,0.0,0.0,1.0
256276,0.0,0.0,1.0
256280,0.0,0.0,1.0
256282,0.0,1.0,0.0
256285,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
256271,0,0,1
256276,0,0,1
256280,0,0,1
256282,0,1,0
256285,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,256271,0,0,1
1,256276,0,0,1
2,256280,0,0,1
3,256282,0,1,0
4,256285,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,256271,,,1
1,256276,,,1
2,256280,,,1
3,256282,,1,
4,256285,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Feb 1 2023 9:15AM,256369,19,"Emersa Waterbox, LLC"
1,Feb 1 2023 9:09AM,256364,16,"SHL Pharma, LLC"
2,Feb 1 2023 9:05AM,256360,15,"Brookfield Pharmaceuticals, LLC"
19,Feb 1 2023 9:00AM,256363,15,"Virtus Pharmaceuticals, LLC"
34,Feb 1 2023 8:57AM,256362,10,ISDIN Corporation
49,Feb 1 2023 8:56AM,256359,15,"Alliance Pharma, Inc."
87,Feb 1 2023 8:51AM,256361,10,"Citieffe, Inc."
88,Feb 1 2023 8:46AM,256358,19,"Graystone, LLC"
90,Feb 1 2023 8:37AM,256357,12,Hush Hush
91,Feb 1 2023 8:30AM,256338,10,"Citieffe, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Feb 1 2023 9:15AM,256369,19,"Emersa Waterbox, LLC",,,1
1,Feb 1 2023 9:09AM,256364,16,"SHL Pharma, LLC",,,1
2,Feb 1 2023 9:05AM,256360,15,"Brookfield Pharmaceuticals, LLC",,,17
3,Feb 1 2023 9:00AM,256363,15,"Virtus Pharmaceuticals, LLC",,,15
4,Feb 1 2023 8:57AM,256362,10,ISDIN Corporation,,,15
5,Feb 1 2023 8:56AM,256359,15,"Alliance Pharma, Inc.",,,38
6,Feb 1 2023 8:51AM,256361,10,"Citieffe, Inc.",,,1
7,Feb 1 2023 8:46AM,256358,19,"Graystone, LLC",,1,1
8,Feb 1 2023 8:37AM,256357,12,Hush Hush,,1,
9,Feb 1 2023 8:30AM,256338,10,"Citieffe, Inc.",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 1 2023 9:15AM,256369,19,"Emersa Waterbox, LLC",1,,
1,Feb 1 2023 9:09AM,256364,16,"SHL Pharma, LLC",1,,
2,Feb 1 2023 9:05AM,256360,15,"Brookfield Pharmaceuticals, LLC",17,,
3,Feb 1 2023 9:00AM,256363,15,"Virtus Pharmaceuticals, LLC",15,,
4,Feb 1 2023 8:57AM,256362,10,ISDIN Corporation,15,,
5,Feb 1 2023 8:56AM,256359,15,"Alliance Pharma, Inc.",38,,
6,Feb 1 2023 8:51AM,256361,10,"Citieffe, Inc.",1,,
7,Feb 1 2023 8:46AM,256358,19,"Graystone, LLC",1,1,
8,Feb 1 2023 8:37AM,256357,12,Hush Hush,,1,
9,Feb 1 2023 8:30AM,256338,10,"Citieffe, Inc.",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 1 2023 9:15AM,256369,19,"Emersa Waterbox, LLC",1,,
1,Feb 1 2023 9:09AM,256364,16,"SHL Pharma, LLC",1,,
2,Feb 1 2023 9:05AM,256360,15,"Brookfield Pharmaceuticals, LLC",17,,
3,Feb 1 2023 9:00AM,256363,15,"Virtus Pharmaceuticals, LLC",15,,
4,Feb 1 2023 8:57AM,256362,10,ISDIN Corporation,15,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 1 2023 9:15AM,256369,19,"Emersa Waterbox, LLC",1.0,NaN,NaN
1,Feb 1 2023 9:09AM,256364,16,"SHL Pharma, LLC",1.0,NaN,NaN
2,Feb 1 2023 9:05AM,256360,15,"Brookfield Pharmaceuticals, LLC",17.0,NaN,NaN
3,Feb 1 2023 9:00AM,256363,15,"Virtus Pharmaceuticals, LLC",15.0,NaN,NaN
4,Feb 1 2023 8:57AM,256362,10,ISDIN Corporation,15.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 1 2023 9:15AM,256369,19,"Emersa Waterbox, LLC",1.0,0.0,0.0
1,Feb 1 2023 9:09AM,256364,16,"SHL Pharma, LLC",1.0,0.0,0.0
2,Feb 1 2023 9:05AM,256360,15,"Brookfield Pharmaceuticals, LLC",17.0,0.0,0.0
3,Feb 1 2023 9:00AM,256363,15,"Virtus Pharmaceuticals, LLC",15.0,0.0,0.0
4,Feb 1 2023 8:57AM,256362,10,ISDIN Corporation,15.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3075843,29.0,15.0,16.0
12,256357,0.0,1.0,0.0
15,769082,70.0,0.0,0.0
16,769003,3.0,0.0,0.0
18,256276,1.0,0.0,0.0
19,2306959,36.0,10.0,8.0
20,256300,0.0,1.0,0.0
21,512633,1.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3075843,29.0,15.0,16.0
1,12,256357,0.0,1.0,0.0
2,15,769082,70.0,0.0,0.0
3,16,769003,3.0,0.0,0.0
4,18,256276,1.0,0.0,0.0
5,19,2306959,36.0,10.0,8.0
6,20,256300,0.0,1.0,0.0
7,21,512633,1.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,29.0,15.0,16.0
1,12,0.0,1.0,0.0
2,15,70.0,0.0,0.0
3,16,3.0,0.0,0.0
4,18,1.0,0.0,0.0
5,19,36.0,10.0,8.0
6,20,0.0,1.0,0.0
7,21,1.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,29.0
1,12,Released,0.0
2,15,Released,70.0
3,16,Released,3.0
4,18,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,18,19,20,21
Status,,,,,,,,
Completed,16.0,0.0,0.0,0.0,0.0,8.0,0.0,0.0
Executing,15.0,1.0,0.0,0.0,0.0,10.0,1.0,1.0
Released,29.0,0.0,70.0,3.0,1.0,36.0,0.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,18,19,20,21
0,Completed,16.0,0.0,0.0,0.0,0.0,8.0,0.0,0.0
1,Executing,15.0,1.0,0.0,0.0,0.0,10.0,1.0,1.0
2,Released,29.0,0.0,70.0,3.0,1.0,36.0,0.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,18,19,20,21
0,Completed,16.0,0.0,0.0,0.0,0.0,8.0,0.0,0.0
1,Executing,15.0,1.0,0.0,0.0,0.0,10.0,1.0,1.0
2,Released,29.0,0.0,70.0,3.0,1.0,36.0,0.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()